In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import utils
import json
from joblib import Parallel, delayed
from time import sleep

from importlib import reload
reload(utils)

<module 'utils' from '/usr/local/google/home/raulramos/projects/text-embeddings-benchmark/utils.py'>

In [3]:
# clone repo https://github.com/amazon-science/esci-data
# previously install git-lfs to download the data files when cloning

ESCI_DATASET_ROOT = '/usr/local/google/home/raulramos/projects/esci-data'

In [4]:
with open('../../secrets/openai-key.txt') as f:
    OPENAI_API_KEY = f.read().strip()


In [5]:
# set locale

locale = 'us'

In [6]:
# load ground truth
dgt = utils.load_examples(ESCI_DATASET_ROOT=ESCI_DATASET_ROOT, locale=locale)

In [7]:
dgt.head()

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color,relevance
0,0,revent 80 cfm,0,B000MOO21W,us,I,0,1,train,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...,None,WhisperCeiling fans feature a totally enclosed...,Panasonic,White,1
1,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train,Homewerks 7141-80 Bathroom Fan Integrated LED ...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,80 CFM,4
2,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train,Homewerks 7140-80 Bathroom Fan Ceiling Mount E...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,White,4
3,3,revent 80 cfm,0,B07RH6Z8KW,us,E,0,1,train,Delta Electronics RAD80L BreezRadiance 80 CFM ...,This pre-owned or refurbished product has been...,Quiet operation at 1.5 sones\nBuilt-in thermos...,DELTA ELECTRONICS (AMERICAS) LTD.,White,4
4,4,revent 80 cfm,0,B07QJ7WYFQ,us,E,0,1,train,Panasonic FV-08VRE2 Ventilation Fan with Reces...,None,The design solution for Fan/light combinations...,Panasonic,White,4


In [8]:
queries = dgt.groupby('query_id').first()[['query']]

In [9]:
products = dgt.groupby('product_id').first()[['product_title', 'product_description', 'product_bullet_point', 'product_brand', 'product_color']]
products = pd.DataFrame([{product_id: utils.get_product_string(pi) for product_id,pi in tqdm(products.iterrows())}], index=['product']).T

1215851it [01:09, 17572.78it/s]


In [10]:
queries.shape, products.shape

((97345, 1), (1215851, 1))

In [11]:
from importlib import reload
reload(utils)

<module 'utils' from '/usr/local/google/home/raulramos/projects/text-embeddings-benchmark/utils.py'>

In [12]:
queries['embeddings'] = utils.get_openai_batch_embeddings(queries['query'].values, OPENAI_API_KEY, chunk_size=500)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 195/195 [07:38<00:00,  2.35s/it]


In [13]:
queries.to_parquet(f'openai/queries-{locale}.parquet')

In [ ]:
products['embeddings'] = utils.get_openai_batch_embeddings(products['product'].values, OPENAI_API_KEY, chunk_size=500)

100%|████████████████████████████████████████████████████████████████████████████████████████████████▉| 2430/2432 [2:02:07<00:06,  3.41s/it]

In [15]:
products.to_parquet(f'openai/products-{locale}.parquet')

In [16]:
products.shape

(1215851, 2)

In [17]:
1

1